# __Constraint Analysis Notebook__

## *This Notebook demonstrates the Constraint analysis procedure for Aircraft Design in its entirety*
###         for 22TTC010 __Aircraft Design__

#### __Step 0:__ importing relevant libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#### **Step 1:** finding atmospheric data
[returns a tuple of relative temperature, pressure and density]

In [ ]:
def Atmos(h):
    '''takes input height in metres and calculates relative temperature, pressure and density'''
    # constants
    R = 287.05827 #m2s-2K-1
    g0 = 9.80665 #ms-2
    T0 = 288.15 #K
    L = -6.5*1e-3 # lapse rate K/m
    Hstar = 11000 # tropopause (boundary separating troposphere from stratosphere)

    if h <= Hstar:
        #relative properties
        theta = 1 +L/T0*h
        delta = (theta)**(-go/(R*L))
        sigma = (theta)**(-g0(R*L)-1)

    else:
        #relative properties
        thetastar = 1 + L/T0+Hstar
        deltastar = (thetastar)**(-g0/(R*L))
        sigmastar = (thetastar)**(-g0/(R*L)-1)
        Tstar =thetastar*T0 #Kelvin

        theta = thetastar
        delta = deltastar
        sigma = sigmastar



    return (theta, delta, sigma)

#### *Physical Constants and Conversions:*

In [ ]:
gamma = 1.4
R = 287.05827 #m2s-2K-1
g0 = 9.80665 #ms-2
T0 = 288.15 #K
P0 = 101325 #Pa
rho0 = P0/(R*T0) #kgm-3

ft2m = 0.3048
kts2ms = 0.5144

#### __Step 2:__ define the military engine model (__Mattingley__)

In [ ]:
def alphaMili(h, Mach, case=1):
    (theta, delta, sigma) = Atmos(h)

    if case == 1: #dry
        return 0.72*(0.88+0.245*(abs(Mach-0.6))**1.4)*sigma**0.7
    elif case == 2: #reheat
        return (0.94+0.38*(Mach-0.5)**2)*sigma**0.7

#### __Step 3:__ define the Master Performance equation

In [ ]:
def Master(WtoS, alpha, beta, q, Cd0, k1, n=1.0, oneoverVdhdt=0, oneovergdVdt=0):
    return (beta/alpha)*(q/(beta*WtoS)*(Cd0+k1*(n*beta/q*WtoS)**2)+oneoverVdhdt+ oneovergdVdt)

#### __Step 4:__ define the Approach Speed equation

In [ ]:
def TslWtoApp(WtoS, beta, rho, CLmax, Vapp):
    Vstall = Vapp/1.3
    q = 0.5*rho*Vstall**2
    WtoSApp = q*CLmax/beta
    #creating a vertical line
    if WtoS < WtoSApp:
        return -50000.0 #arbitrarily large figure
    else:
        return +50000.0

#### __Step 5:__ define the Take-off Distance equation

In [ ]:
def TslWtoTO(WtoS, alpha, beta, rho, CLmax, mu, g, kL, sL):
    return ((beta**2/alpha*kL0**2/sG*rho*g*CLmaxL0)*WtoS)

#### __Step 6:__ define the Landing Distance equation

In [ ]:
def TslWtoLand(WtoS, beta, rho, CLmax, mu, g, kL, sL):
    WtoSLand = sL*mu*rho*CLmax/(beta*kL**2)
    #creating a vertical line
    if WtoS < WtoSLand:
        return -50000.0
    else:
        return +50000.0

#### __Step 7:__ defining Aerodynamic assumptions

In [ ]:
Cd0 = 0.015
Cd0Sup = 2.0*Cd0
AR = 3.0
wingSweepDeg = 42.0 #LE
wingSweepRad = np.radians(wingSweepDeg)

e - 4.61*(10.0-0.045*AR**0.68)*(np.cos(wingSweepRad))**0.15 - 3.1 #Oswald effiiciency factor for the fighter model
k1 = 1.0/(np.pi*e*AR)

print('e:', e, 'k1:', k1)

def k1SupRaymer(AR, Mach, LESweepRad):
    MsqM1 = (Mach**2 - 1.0)
    return AR*MsqM1/(4.0*AR*np.sqrt(MsqM1) - 2.0)*np,cos(LESweepRad)

print('k1 (supersonic):', k1SupRaymer(AR,1.5,wingSweepRad))

CLmaxL0 = 1.24
CLmax   = 1.24

print('CLmaxL0:', CLmaxL0, 'CLmax:', CLmax)

#### *Ranges to Compute and Plot Thrust-to-Weight and Wing Loading ratios:* 

In [ ]:
minWtoS = 200.0; maxWtoS = 600.0; incWtoS = 50.0
minTslWto = 0.2; maxTslWto = 1.6
WtoS = np.arrang(minWtoS, maxWtoS, incWtoS)
len = WtoS.size
TslWto = np.zeros(len)
TslWtoPlot = np.zeros(len)

#### *Take-Off Distance = 214m*

In [ ]:
name = 'take-off distance 214m'

alpha = alphaMili(0, 0.0, case=2) #take-off afterburner on
print('take-off alpha:', alpha)

beta = 1.0
(theta, delta, sigma) = Atmos(0.0) #at sea level
rho = sigma*rho0
kL0 = 1.1 #lift-off speed factor
sG = 214 #take-off field length

In [ ]:
for i in range(0,len):
    TslWto[i] = TslWtoTO(WtoS[i], alpha, beta, rho, CLmaxL0, g0, kL0, sG)

In [ ]:
TslWtoPlot = np.copy(TslWto) #first constraint
constraintName = [name]

In [ ]:
#example constraint line
plt.xlabel(r'$W_{to}/S$ [Pa]')
plt.ylabel(r'$T_{sl}/W_{to}$')
plt.title('Constraint Diagram')
plt.axis([minWtoS, maxWtoS, minTslWto, maxTslWto])
plt.grid(True)
plt.plot(WtoS, TslWto, 'b', label=name)
plt.legend()
plt.savefig('T214.pdf')

#### *Take-Off Distance = 428m*

In [ ]:
name = 'take-off distance 428m'

alpha = alphaMili(0, 0.0, case=2) #take-off afterburner on
print('take-off alpha:', alpha)

beta = 1.0
(thera, delta, sigma) = Atmos(0.0) #sea level
rho = sigma*rho0

kL0 = 1.1 #lift-off speed factor
sG = 450 #take-off field length [m]

In [ ]:
for i in range(0,len):
    TslWto[i] = TslWtoTO(WtoS[i], alpha, beta, rho, CLmaxL0, g0, kL0, sG)

In [ ]:
TslWtoPlot = np.vstack([TslWtoPlot, TslWto]) #adding row
constraintName.append(name)

In [ ]:
#exaple constraint line
plt.xlabel(r'$W_{to}/S$ [Pa]')
plt.ylabel(r'$T_{sl}/W_{to}$')
plt.title('Constraint Diagram')
plt.axis([minWtoS, maxWtoS, minTslWto, maxTslWto])
plt.grid(True)
plt.plot(WtoS, TslWto, 'b', label=name)
plt.legend()
plt.savefig('T0428.pdf')

#### __Step 8:__ setting conditions for Supersonic Cruise

In [ ]:
name = 'Supersonic Cruise'

height = 5000*ft2m
Mach = 1.5
alpha = alphaMili(height, Mach, case=1) #afterburner off
beta = 0.85

(theta, delta, sigma) = Atmos(height)
TK = theta*T0
p = delta*P0
rho = sigma*rho0
a = np.sqrt(gamma*R*TK)

fpm = 100.0
climb = 1/(Mach*a)*fpm*ft2m/60.0
k1Sup = k1SupRaymer(AR,Mach,wingSweepRad)

In [ ]:
for i in range(0,len):
    TslWto[i] = Master(WtoS[i], alpha, beta, q, Cd0Sup, k1Sup, n=1.0,oneoverVdhdt=climb, oneovergdVdt=0.0)

In [ ]:
TslWtoPlot = np.stack([TslWtoPlot, TslWto]) #adding another row
constraintName.append(name)

In [ ]:
#example constraint line
plt.xlabel(r'$W_{to}/S$ [Pa]')
plt.ylabel(r'$T_{sl}/W_{to}$')
plt.title('Constraint Diagram')
plt.axis([minWtoS, maxWtoS, minTslWto, maxTslWto])
plt.grid(True)
plt.plot(WtoS, TslWto, 'b', label=name)
plt.legend()
plt.savefig('SupersonicCruise.pdf')

#### __Step 9:__ setting conditions for the Supersonic Turn

In [ ]:
name = 'Supersonic Turn'

height = 30000*ft2m
Mach = 1.5
alpha = alphaMili(height, Mach, 2) #afterburner on
beta = 0.80

(theta, delta, sigma,) = Atmos(height)
TK = theta*T0
p = delta*P0
rho = sigma*rho0
a = np.sqrt(gamma*R*TK)
q = 0.5*rho*(Mach*a)**2

k1Sup = k1SupRaymer(AR, Mach, wingSweepRad)

In [ ]:
for i in range(0,len):
    TslWto[i] = Master(WtoS[i], alpha, beta, q, Cd0Sup, k1Sup, n=5.0, oneoverVdhdt=0.0, oneovergdVdt=0.0)

In [ ]:
TslWtoPlot = np.stack([TslWtoPlot, TslWto]) #adding another row
constraintName.append(name)

In [ ]:
#example constraint line
plt.xlabel(r'$W_{to}/S$ [Pa]')
plt.ylabel(r'$T_{sl}/W_{to}$')
plt.title('Constraint Diagram')
plt.axis([minWtoS, maxWtoS, minTslWto, maxTslWto])
plt.grid(True)
plt.plot(WtoS, TslWto, 'b', label=name)
plt.legend()
plt.savefig('SupersonicTurn.pdf')

#### __Step 10:__ setting conditions for the Subsonic Turn

In [ ]:
name = 'Subsonic Turn'

height = 15000*ft2m
Mach = 0.9
alpha = alphaMili(height, Mach, 2) #afterburner on
beta = 0.75

(theta, delta, sigma) = Atmos(height)
TK = theta*T0
p = delta*P0
rho = sigma*rho0
a = np.sqrt(gamma*R*TK)
q = 0.5*rho*(Mach*a)**2

In [ ]:
for i in range(0,len):
    TslWto[i] = Master(WtoS[i], alpha, beta, q, Cd0, k1, n=8.0, oneoverVdhdt=0.0, oneovergdVdt=0.0)

In [ ]:
TslWtoPlot = np.stack([TslWtoPlot, TslWto]) #adding another row
constraintName.append(name)

In [ ]:
#example constraint line
plt.xlabel(r'$W_{to}/S$ [Pa]')
plt.ylabel(r'$T_{sl}/W_{to}$')
plt.title('Constraint Diagram')
plt.axis([minWtoS, maxWtoS, minTslWto, maxTslWto])
plt.grid(True)
plt.plot(WtoS, TslWto, 'b', label=name)
plt.legend()
plt.savefig('SubsonicTurn.pdf')

#### __Step 11:__ setting conditions for Landing 

In [ ]:
name = 'Landing Distance 450m'
beta = 0.75 # max landing weight ratio
(theta, delta, sigma) = Atmos(0.0) #sea level
rho = sigma*rho0
mu = 0.5
kL = 1.10 #speed at flair
sL  = 450 #ground roll

In [ ]:
for i in range(0,len):
    TslWto[i] = Master(WtoS[i], beta, rho, CLmax, mu, g0, kL, sL)

In [ ]:
# store in plot
TslWtoPlot = np.vstack([TslWtoPlot, TslWto]) #add row
constraintName.append(name)

In [ ]:
#example constraint line
plt.xlabel(r'$W_{to}/S$ [Pa]')
plt.ylabel(r'$T_{sl}/W_{to}$')
plt.title('Constraint Diagram')
plt.axis([minWtoS, maxWtoS, minTslWto, maxTslWto])
plt.grid(True)
plt.plot(WtoS, TslWto, 'b', label=name)
plt.legend()
plt.savefig('Landing.pdf')

#### __Step 12:__ finding the Design Point

In [ ]:
# find max at each wing loading and put it back in TslWto, used for design point DP
TslWto = TslWtoPlot.max(axis=0)
DPTW = TslWto.min(); loc = TslWto.argmin(); DPWS = WtoS[loc];
DPTW *=1.05; DPWS *= 0.98 #adding some small margin, higher T/W, lower wing loading
print('Design Point: Wing Loading:', DPWS, 'Pa, T/W:', DPTW)

#### __Step 13:__ drawing the final Constraint Diagram

In [ ]:
plt.xlabel(r'$W_{to}/S$ [Pa]')
plt.ylabel(r'$T_{sl}/W_{to}$')
plt.title('Constraint Diagram')
plt.axis([minWtoS, maxWtoS, minTslWto, maxTslWto])
plt.grid(True)
id = 0

for row in TslWtoPlot:
    plt.plot(WtoS, row, label=constraintName[id])
    id += 1
plot.plot(WtoS, TslWto, '--', linewidth=3.0, label='design space')
plt.scatter(DPWS, DPTW, alpha=0.3, s=150, c='red', label='design point')

# use pandas library to read spreadsheet 
benchmark = pd.read_csv('benchmarkaircrafthaf.csv')

WSBench = benchmark['Wing Loading (Pa)']; TWBench = benchmark['T/W']
plt.scatter(WSBench, TWBench, alpha=0.5, s=100, c='blue', label='benchmark a/c')

plt.legend(loc='upper right')
plt.savefig('interceptorconstraint.pdf')

In [ ]:
benchmark.head(4)